In [1]:
import requests
import pandas as pd
from pathlib import Path

# 1) paste the JSON endpoint you saw in DevTools (from "Copy as cURL")
#    Examples you might see (pattern varies by station/dashboard view):
#    - https://ambientweather.net/dashboard/<stationID>/tiles?start=...&end=...
#    - https://rt.ambientweather.net/v1/devices/<deviceId>/data?start=...&end=...&limit=...
#    - another /api/... path returning arrays of observations

ENDPOINT = "https://ambientweather.net/dashboard/<stationID>/tiles?start=<unix>&end=<unix>"

ENDPOINT = "https://ambientweather.net/dashboard/<fd3b1c6ad8b91db6d41b3e54873480e4>/tiles?start=<unix>&end=<unix>"

# 2) set a Referer to mimic the dashboard (often not strictly required, but makes it robust)
STATION_DASHBOARD = "https://ambientweather.net/dashboard/<stationID>"

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": STATION_DASHBOARD,
}

r = requests.get(ENDPOINT, headers=headers, timeout=30)
r.raise_for_status()
data = r.json()

# If it's a list of observations, normalize directly:
def normalize(obj):
    # Handles both list payloads and dicts with a 'data' field
    if isinstance(obj, list):
        return pd.json_normalize(obj)
    if isinstance(obj, dict):
        # try common keys
        for k in ("data", "observations", "items", "result"):
            if k in obj and isinstance(obj[k], (list, dict)):
                return pd.json_normalize(obj[k])
        return pd.json_normalize(obj)
    return pd.DataFrame()

df = normalize(data)

# Optional: convert epoch seconds to pandas datetime if you see ts fields
for col in df.columns:
    if col.lower() in {"date", "time", "timestamp", "ts"} or col.lower().endswith("time"):
        # try to parse as epoch seconds
        try:
            df[col] = pd.to_datetime(df[col], unit="s", errors="ignore")
        except Exception:
            pass

# Save
out = Path("ambient_export.csv")
# df.to_csv(out, index=False)
# print(f"Saved {len(df):,} rows to {out.resolve()}")


HTTPError: 404 Client Error: Not Found for url: https://ambientweather.net/dashboard/%3CstationID%3E/tiles?start=%3Cunix%3E&end=%3Cunix%3E

In [2]:
ENDPOINT = "https://ambientweather.net/dashboard/fd3b1c6ad8b91db6d41b3e54873480e4/tiles?start=1724371200&end=1724457600"

# 2) set a Referer to mimic the dashboard (often not strictly required, but makes it robust)
STATION_DASHBOARD = "https://ambientweather.net/dashboard/<stationID>"

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": STATION_DASHBOARD,
}

r = requests.get(ENDPOINT, headers=headers, timeout=30)
r.raise_for_status()
data = r.json()

HTTPError: 404 Client Error: Not Found for url: https://ambientweather.net/dashboard/fd3b1c6ad8b91db6d41b3e54873480e4/tiles?start=1724371200&end=1724457600

In [3]:
import requests, json

ROOT = "https://lightning.ambientweather.net/devices/6686ec9fe2090472957473f5"
headers = {
    "User-Agent": "Mozilla/5.0",
    # Referer usually not required here, but harmless to include:
    "Referer": "https://ambientweather.net/",
}

r = requests.get(ROOT, headers=headers, timeout=30)
print("status:", r.status_code)
print(r.headers.get("content-type"))
print(json.dumps(r.json(), indent=2)[:2000])  # preview first ~2k chars


status: 200
application/json; charset=utf-8
{
  "_id": "6686ec9fe2090472957473f5",
  "macAddress": "C8:C9:A3:10:32:1D",
  "lastData": {
    "stationtype": "AMBWeatherPro_V5.1.1",
    "dateutc": 1756041360000,
    "tempf": 86.7,
    "humidity": 80,
    "windspeedmph": 0.22,
    "windgustmph": 1.12,
    "maxdailygust": 9.17,
    "winddir": 287,
    "uv": 3,
    "solarradiation": 320.04,
    "hourlyrainin": 0,
    "eventrainin": 3.571,
    "dailyrainin": 0,
    "weeklyrainin": 0,
    "monthlyrainin": 9.799,
    "yearlyrainin": 32.26,
    "totalrainin": 32.26,
    "baromrelin": 29.772,
    "baromabsin": 29.982,
    "type": "weather-data",
    "created_at": 1756041365720,
    "feelsLike": 101.98641838029994,
    "dateutc5": 1756041300000,
    "lastRain": 1755978300000,
    "discreets": {
      "humidity1": [
        58,
        57
      ]
    },
    "tz": "America/New_York",
    "hl": {
      "dateutc": 1756008000000,
      "dewPoint": {
        "h": 79.76877877486689,
        "l": 76.45147

In [4]:
import time
import math
import requests
import pandas as pd
from datetime import datetime, timezone, timedelta

BASE = "https://lightning.ambientweather.net/device-data"
MAC  = "C8:C9:A3:10:32:1D"  # your macAddress (plain; requests will URL-encode)

HEADERS = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://ambientweather.net/",  # usually not required but helps
}

def to_ms(dt):
    # dt as aware UTC datetime → epoch ms
    return int(dt.replace(tzinfo=timezone.utc).timestamp() * 1000)

def fetch_chunk(start_dt, end_dt, data_key="yesterdayData", limit=100, asc=True):
    params = {
        "macAddress": MAC,
        "dataKey": data_key,
        "start": to_ms(start_dt),
        "end": to_ms(end_dt),
        "limit": limit,
        "asc": str(asc).lower(),
    }
    r = requests.get(BASE, params=params, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return r.json()

today_utc = datetime.now(timezone.utc).date()
y_start = datetime.combine(today_utc - timedelta(days=1), datetime.min.time(), tzinfo=timezone.utc)
y_end   = y_start + timedelta(days=1)
payload = fetch_chunk(y_start, y_end, data_key="yesterdayData", limit=100, asc=True)


In [ ]:
import time
import math
import requests
import pandas as pd
from datetime import datetime, timezone, timedelta

BASE = "https://lightning.ambientweather.net/device-data"
MAC  = "C8:C9:A3:10:32:1D"  # your macAddress (plain; requests will URL-encode)

HEADERS = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://ambientweather.net/",  # usually not required but helps
}

def to_ms(dt):
    # dt as aware UTC datetime → epoch ms
    return int(dt.replace(tzinfo=timezone.utc).timestamp() * 1000)

def fetch_chunk(start_dt, end_dt, limit=100, asc=True):
    params = {
        "macAddress": MAC,
        "start": to_ms(start_dt),
        "end": to_ms(end_dt),
        "limit": limit,
        "asc": str(asc).lower(),
    }
    r = requests.get(BASE, params=params, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return r.json()

today_utc = datetime.now(timezone.utc).date()
y_start = datetime.combine(today_utc - timedelta(days=7), datetime.min.time(), tzinfo=timezone.utc)
y_end   = y_start + timedelta(days=1)
payload = fetch_chunk(y_start, y_end, limit=1000, asc=True)
len(payload['data'])

In [15]:
import time
import math
import requests
import pandas as pd
from datetime import datetime, timezone, timedelta

BASE = "https://lightning.ambientweather.net/device-data"
MAC  = "C8:C9:A3:10:32:1D"  # your macAddress (plain; requests will URL-encode)

HEADERS = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://ambientweather.net/",  # usually not required but helps
}

def to_ms(dt):
    # dt as aware UTC datetime → epoch ms
    return int(dt.replace(tzinfo=timezone.utc).timestamp() * 1000)

def fetch_chunk(start_dt, end_dt, limit=100, asc=True):
    params = {
        "macAddress": MAC,
        "start": to_ms(start_dt),
        "end": to_ms(end_dt),
        "limit": limit,
        "asc": str(asc).lower(),
    }
    r = requests.get(BASE, params=params, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return r.json()

today_utc = datetime.now(timezone.utc).date()
y_start = datetime.combine(today_utc - timedelta(days=365), datetime.min.time(), tzinfo=timezone.utc)
y_end   = y_start + timedelta(days=1)
payload = fetch_chunk(y_start, y_end, limit=1000, asc=True)
len(payload['data'])

289

In [16]:
payload['data']

[{'_id': '66c922aa8564b0825f3d8870',
  'PASSKEY': '94087a0fb69bb1ddc81050b863291c31',
  'stationtype': 'AMBWeatherPro_V5.1.1',
  'dateutc': 1724457600000,
  'tempf': 78.1,
  'humidity': 85,
  'windspeedmph': 7.83,
  'windgustmph': 10.29,
  'maxdailygust': 22.82,
  'winddir': 138,
  'uv': 0,
  'solarradiation': 4.43,
  'hourlyrainin': 0,
  'eventrainin': 0,
  'dailyrainin': 0,
  'weeklyrainin': 1.39,
  'monthlyrainin': 1.39,
  'yearlyrainin': 1.39,
  'totalrainin': 1.39,
  'baromrelin': 30.053,
  'baromabsin': 30.127,
  'pm25': 2,
  'pm25_24h': 3.6,
  'type': 'weather-data',
  'created_at': 1724457641821,
  'ip': '10.138.138.193',
  'feelsLike': 79.605,
  'dateutc5': 1724457600000,
  'lastRain': 1724353680000,
  'deviceId': '6686ec9fe2090472957473f5',
  'passkey': '94087a0fb69bb1ddc81050b863291c31',
  'time': 1724457600000,
  'loc': '94087a0fb69bb1ddc81050b863291c31/daily/1724457600000.json'},
 {'_id': '66c923dbb02b312bc3f77b9c',
  'PASSKEY': '94087a0fb69bb1ddc81050b863291c31',
  'stati

In [18]:
import time, random
import requests, pandas as pd
from datetime import datetime, timezone, timedelta
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

BASE = "https://lightning.ambientweather.net/device-data"
MAC  = "C8:C9:A3:10:32:1D"

HEADERS = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://ambientweather.net/",
    "Accept": "application/json, text/plain, */*",
    "Connection": "keep-alive",     # you can also try "close" if drops persist
}

def to_ms(dt):
    return int(dt.replace(tzinfo=timezone.utc).timestamp() * 1000)

# --- robust session with retries/backoff ---
session = requests.Session()
retry = Retry(
    total=8,
    connect=5,
    read=5,
    status=5,
    backoff_factor=0.5,                 # 0.5,1,2,4,8s...
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"],
    raise_on_status=False,
)
session.mount("https://", HTTPAdapter(max_retries=retry))

def fetch_chunk(start_dt, end_dt, asc=True):
    params = {
        "macAddress": MAC,
        "start": to_ms(start_dt),
        "end": to_ms(end_dt),
        "asc": str(asc).lower(),
    }
    # small jitter to avoid hammering
    time.sleep(0.2 + random.random() * 0.4)
    r = session.get(BASE, params=params, headers=HEADERS, timeout=(5, 30))
    # If server closed w/out response, requests+Retry will auto-retry. Still check:
    r.raise_for_status()
    return r.json()

def normalize(payload):
    rows = payload.get("data", payload if isinstance(payload, list) else [])
    df = pd.json_normalize(rows)
    # convert ms → datetime for any likely time fields
    for c in df.columns:
        lc = c.lower()
        if lc in {"time","timestamp","date","ts"} or lc.endswith("time"):
            try:
                df[c] = pd.to_datetime(df[c], unit="ms", utc=True)
            except Exception:
                pass
    return df

# ---- pull range in smaller windows (6h) ----
end_dt   = datetime.now(timezone.utc)
start_dt = end_dt - timedelta(days=365)
step     = timedelta(hours=6)           # 3–6h is usually very stable

frames = []
t0 = start_dt
while t0 < end_dt:
    t1 = min(t0 + step, end_dt)
    try:
        payload = fetch_chunk(t0, t1, asc=True)
        df = normalize(payload)
        if not df.empty:
            frames.append(df)
    except requests.exceptions.HTTPError as e:
        # print minimal diagnostics and continue
        print(f"[{t0:%Y-%m-%d %H:%M}] HTTPError {e.response.status_code} – {e.response.text[:150]}")
    except requests.exceptions.RequestException as e:
        # covers RemoteDisconnected/ConnectionError etc.
        print(f"[{t0:%Y-%m-%d %H:%M}] transient error: {e}")
        # brief pause before next window
        time.sleep(2)
    t0 = t1

hist = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
ts_col = next((c for c in hist.columns if c.lower() in {"time","timestamp","date","ts"} or c.lower().endswith("time")), None)
if ts_col:
    hist.drop_duplicates(subset=[ts_col], inplace=True)
    hist.sort_values(by=ts_col, inplace=True)

hist.to_csv("ambient_history.csv", index=False)
print("saved", len(hist), "rows")


saved 332 rows


In [19]:
hist

,_id,PASSKEY,stationtype,dateutc,tempf,humidity,windspeedmph,windgustmph,maxdailygust,winddir,...,type,created_at,ip,feelsLike,dateutc5,lastRain,deviceId,passkey,time,loc
0,66c9e77d73984bb4b4fbb034,94087a0fb69bb1ddc81050b863291c31,AMBWeatherPro_V5.1.1,1724508000000,81.9,82,4.92,5.82,11.41,146,...,weather-data,1724508028697,172.71.154.17,89.047797,1724508000000,1724353680000,6686ec9fe2090472957473f5,94087a0fb69bb1ddc81050b863291c31,2024-08-24 00:00:00+00:00,94087a0fb69bb1ddc81050b863291c31/daily/1724457...
120,66ca7414a9ebbe907ee92691,94087a0fb69bb1ddc81050b863291c31,AMBWeatherPro_V5.1.1,1724544000000,86.9,69,3.13,4.47,11.41,344,...,weather-data,1724544020136,10.138.234.211,96.832887,1724544000000,1724353680000,6686ec9fe2090472957473f5,94087a0fb69bb1ddc81050b863291c31,2024-08-25 00:00:00+00:00,94087a0fb69bb1ddc81050b863291c31/daily/1724544...
408,66cbc5bd9e68b9dddf5a2ce2,94087a0fb69bb1ddc81050b863291c31,AMBWeatherPro_V5.1.1,1724630460000,78.6,96,1.79,2.24,17.22,206,...,weather-data,1724630460393,172.71.154.137,80.672000,1724630400000,1724630460000,6686ec9fe2090472957473f5,94087a0fb69bb1ddc81050b863291c31,2024-08-26 00:00:00+00:00,94087a0fb69bb1ddc81050b863291c31/daily/1724630...
696,66cd17275cbc7eb958c9449a,94087a0fb69bb1ddc81050b863291c31,AMBWeatherPro_V5.1.1,1724716800000,75.4,99,7.38,8.05,28.63,67,...,weather-data,1724716838288,10.138.138.193,77.293000,1724716800000,1724716800000,6686ec9fe2090472957473f5,94087a0fb69bb1ddc81050b863291c31,2024-08-27 00:00:00+00:00,94087a0fb69bb1ddc81050b863291c31/daily/1724716...
984,66ce6892ce08dbe9aa0d606c,94087a0fb69bb1ddc81050b863291c31,AMBWeatherPro_V5.1.1,1724803200000,74.1,97,1.34,2.24,19.46,346,...,weather-data,1724803217420,10.138.80.59,75.769000,1724803200000,1724803200000,6686ec9fe2090472957473f5,94087a0fb69bb1ddc81050b863291c31,2024-08-28 00:00:00+00:00,94087a0fb69bb1ddc81050b863291c31/daily/1724803...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47282,68a26d35a7468059102965a6,94087a0fb69bb1ddc81050b863291c31,AMBWeatherPro_V5.1.1,1755475200000,86.4,78,1.79,2.24,15.88,305,...,weather-data,1755475253647,172.56.73.189,99.978762,1755475200000,1755314100000,6686ec9fe2090472957473f5,94087a0fb69bb1ddc81050b863291c31,2025-08-18 00:00:00+00:00,94087a0fb69bb1ddc81050b863291c31/daily/1755475...
47412,68a3be9638123f7e554a9d5b,94087a0fb69bb1ddc81050b863291c31,AMBWeatherPro_V5.1.1,1755561600000,80.1,97,0.00,0.00,20.58,294,...,weather-data,1755561609909,172.56.77.246,88.692089,1755561600000,1755559800000,6686ec9fe2090472957473f5,94087a0fb69bb1ddc81050b863291c31,2025-08-19 00:00:00+00:00,94087a0fb69bb1ddc81050b863291c31/daily/1755561...
47547,68a510049a012f132a83cbe6,94087a0fb69bb1ddc81050b863291c31,AMBWeatherPro_V5.1.1,1755648000000,87.3,74,0.67,1.12,12.53,342,...,weather-data,1755648004530,172.56.79.136,100.472611,1755648000000,1755634560000,6686ec9fe2090472957473f5,94087a0fb69bb1ddc81050b863291c31,2025-08-20 00:00:00+00:00,94087a0fb69bb1ddc81050b863291c31/daily/1755648...
47692,68a66182eb495154ae8f287a,94087a0fb69bb1ddc81050b863291c31,AMBWeatherPro_V5.1.1,1755734400000,85.5,74,1.34,2.24,12.53,282,...,weather-data,1755734402244,172.56.73.254,95.550184,1755734400000,1755634560000,6686ec9fe2090472957473f5,94087a0fb69bb1ddc81050b863291c31,2025-08-21 00:00:00+00:00,94087a0fb69bb1ddc81050b863291c31/daily/1755734...


In [ ]:
import time, random
import requests, pandas as pd
from datetime import datetime, timezone, timedelta
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

BASE = "https://lightning.ambientweather.net/device-data"
MAC  = "C8:C9:A3:10:32:1D"

HEADERS = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://ambientweather.net/",
    "Accept": "application/json, text/plain, */*",
    "Connection": "keep-alive",     # you can also try "close" if drops persist
}

def to_ms(dt):
    return int(dt.replace(tzinfo=timezone.utc).timestamp() * 1000)

# --- robust session with retries/backoff ---
session = requests.Session()
retry = Retry(
    total=8,
    connect=5,
    read=5,
    status=5,
    backoff_factor=0.5,                 # 0.5,1,2,4,8s...
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"],
    raise_on_status=False,
)
session.mount("https://", HTTPAdapter(max_retries=retry))

def fetch_chunk(start_dt, end_dt, asc=True):
    params = {
        "macAddress": MAC,
        "start": to_ms(start_dt),
        "end": to_ms(end_dt),
        "asc": str(asc).lower(),
    }
    # small jitter to avoid hammering
    time.sleep(0.2 + random.random() * 0.4)
    r = session.get(BASE, params=params, headers=HEADERS, timeout=(5, 30))
    # If server closed w/out response, requests+Retry will auto-retry. Still check:
    r.raise_for_status()
    return r.json()

def normalize(payload):
    rows = payload.get("data", payload if isinstance(payload, list) else [])
    df = pd.json_normalize(rows)
    # convert ms → datetime for any likely time fields
    for c in df.columns:
        lc = c.lower()
        if lc in {"time","timestamp","date","ts"} or lc.endswith("time"):
            try:
                df[c] = pd.to_datetime(df[c], unit="ms", utc=True)
            except Exception:
                pass
    return df

# ---- pull range in smaller windows (6h) ----
end_dt   = datetime.now(timezone.utc)
start_dt = end_dt - timedelta(days=365)
step     = timedelta(hours=6)           # 3–6h is usually very stable

frames = []
t0 = start_dt
while t0 < end_dt:
    t1 = min(t0 + step, end_dt)
    try:
        payload = fetch_chunk(t0, t1, asc=True)
        df = normalize(payload)
        if not df.empty:
            frames.append(df)
    except requests.exceptions.HTTPError as e:
        # print minimal diagnostics and continue
        print(f"[{t0:%Y-%m-%d %H:%M}] HTTPError {e.response.status_code} – {e.response.text[:150]}")
    except requests.exceptions.RequestException as e:
        # covers RemoteDisconnected/ConnectionError etc.
        print(f"[{t0:%Y-%m-%d %H:%M}] transient error: {e}")
        # brief pause before next window
        time.sleep(2)
    t0 = t1

hist = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
ts_col = next((c for c in hist.columns if c.lower() in {"time","timestamp","date","ts"} or c.lower().endswith("time")), None)
if ts_col:
    hist.drop_duplicates(subset=[ts_col], inplace=True)
    hist.sort_values(by=ts_col, inplace=True)